# XMM-Newton Timing Analysis: CTCV J2056-3014

Period searches for 2019 and 2022 XMM-Newton observations.

## XMM-Newton-Specific Notes

XMM-Newton carries three co-aligned X-ray telescopes with CCD cameras: one PN (higher effective area, faster readout) and two MOS detectors (MOS1, MOS2). Key considerations:

- **Separate detector analysis**: Each detector (PN, MOS1, MOS2) must be analyzed independently due to different effective areas, backgrounds, and timing properties. Results can be combined afterward.
- **Background extraction**: Unlike NICER's model-based background, XMM uses **circular extraction regions** on the detector. Source counts come from a region centered on the target; background from a source-free region on the same chip. The background must be scaled by the ratio of extraction areas.
- **PN dominates**: The PN detector typically provides ~2-3x more counts than each MOS due to its larger effective area, especially important for timing analysis.

In [22]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')

import numpy as np
from stingray import EventList, Lightcurve
from stingray.pulse import z_n_search
from astropy.time import Time, TimeDelta
import hendrics.io as HENio

from bokeh.plotting import figure, show
from bokeh.layouts import column, row, gridplot
from bokeh.io import output_notebook
from bokeh.models import Title

from timing_analysis import *
from plotting import *
from statistics import *

output_notebook()

Loading BokehJS ...

## Load Data

In [4]:
events = {
    '0842570101': {'pn':{'src':{}, 'bkg':{}}, 'mos1':{'src':{}, 'bkg':{}}, 'mos2':{'src':{}, 'bkg':{}}},
    '0902500101': {'pn':{'src':{}, 'bkg':{}}, 'mos1':{'src':{}, 'bkg':{}}, 'mos2':{'src':{}, 'bkg':{}}},
}

data_path = '../../../xmm'

for obs in events:
    for module in ['pn', 'mos1', 'mos2']:
        events[obs][module]['src']['total'] = HENio.load_events(f'{data_path}/{obs}/{module}_clean_src.evt')
        events[obs][module]['bkg']['total'] = HENio.load_events(f'{data_path}/{obs}/{module}_clean_bkg.evt')

# Energy filtering
for obs in events:
    for module in events[obs]:
        for region in events[obs][module]:
            events[obs][module][region]['total'].filter_energy_range([0.3, 10], inplace=True)
            events[obs][module][region]['soft'] = events[obs][module][region]['total'].filter_energy_range([0.3, 2])
            events[obs][module][region]['hard'] = events[obs][module][region]['total'].filter_energy_range([2, 10])

# Create lightcurves
lightcurves = {}
for obs in events:
    lightcurves[obs] = {}
    for module in events[obs]:
        lightcurves[obs][module] = {}
        for region in events[obs][module]:
            lightcurves[obs][module][region] = {}
            for energy in events[obs][module]['src']:
                lightcurves[obs][module][region][energy] = {}
                for dt in [1, 5, 20, 50]:
                    lightcurves[obs][module][region][energy][dt] = events[obs][module][region][energy].to_lc(dt=dt)

/opt/miniconda3/envs/CTCV_PINT/lib/python3.13/site-packages/stingray/io.py:1044: UserWarning: No valid GTI extensions found. 
Error: list index out of range
GTIs will be set to the entire time series.
  warnings.warn(


## Observation Summary

In [5]:
print("XMM OBSERVATION SUMMARY\n")

for obs in events:
    factor = (27**2)/(357**2) if obs == '0842570101' else (20**2)/(113**2)
    
    print(f"OBSERVATION: {obs}")
    
    total_src_counts = 0
    total_bkg_scaled = 0
    total_net_counts = 0
    total_net_rate = 0
    total_time_avg = 0
    
    for module in events[obs]:
        src_counts = len(events[obs][module]['src']['total'].time)
        bkg_counts = len(events[obs][module]['bkg']['total'].time)
        bkg_scaled = bkg_counts * factor
        net_source = src_counts - bkg_scaled
        
        total_time = np.sum(events[obs][module]['src']['total'].gti[:, 1] - events[obs][module]['src']['total'].gti[:, 0])
        
        print(f"  {module.upper():4s}  Time: {total_time/1000:6.2f} ks  |  Source: {src_counts:7,.0f}  Bkg: {bkg_scaled:7.1f}  Net: {net_source:7.1f}")
        
        total_src_counts += src_counts
        total_bkg_scaled += bkg_scaled
        total_net_counts += net_source
        total_time_avg += total_time
    
    avg_time = total_time_avg / len(events[obs])
    print(f"  TOTAL Time: {avg_time/1000:6.2f} ks  |  Source: {total_src_counts:7,.0f}  Bkg: {total_bkg_scaled:7.1f}  Net: {total_net_counts:7.1f}")
    print()

XMM OBSERVATION SUMMARY

OBSERVATION: 0842570101
  PN    Time:  15.55 ks  |  Source:   5,179  Bkg:    77.5  Net:  5101.5
  MOS1  Time:  17.60 ks  |  Source:   2,930  Bkg:    26.8  Net:  2903.2
  MOS2  Time:  17.58 ks  |  Source:   3,038  Bkg:     8.5  Net:  3029.5
  TOTAL Time:  16.91 ks  |  Source:  11,147  Bkg:   112.8  Net: 11034.2

OBSERVATION: 0902500101
  PN    Time:  51.16 ks  |  Source:  15,380  Bkg:   128.3  Net: 15251.7
  MOS1  Time:  52.99 ks  |  Source:   5,945  Bkg:    97.9  Net:  5847.1
  MOS2  Time:  52.98 ks  |  Source:   6,203  Bkg:    91.0  Net:  6112.0
  TOTAL Time:  52.38 ks  |  Source:  27,528  Bkg:   317.2  Net: 27210.8



## Lightcurve Visualization

In [ ]:
plots = []
binsize = 50

for obs in lightcurves:
    p = figure(
        width=1100, height=300,
        title=Title(text=f"{obs}", text_font_size="18pt", align="center"),
        x_axis_label='Time (UTC)',
        y_axis_label='Countrate (cts/s)',
        x_axis_type='datetime'
    )
    set_axis_styles(p)
    
    lc = lightcurves[obs]['pn']['src']['total'][binsize]
    plot_lightcurve(p, lc, time_format='utc', color="#1F77B4") # Plot using default mpl color
    plots.append(p)

show(column(plots))

## Z^2 Periodogram

Period search using the Z^2 statistic on unbinned event times. Each detector module is analyzed separately to verify consistent period detection across all three cameras.

In [23]:
f_min, f_max = 0.0335, 0.034
frequencies = np.linspace(f_min, f_max, 4096)
nharm = 1

Z2 = {}
plots = []

for obs in lightcurves:
    Z2[obs] = {}
    for module in lightcurves[obs]:
        p = figure(width=500, height=400,
                   x_axis_label='Frequency (Hz)',
                   y_axis_label='Power')
        set_axis_styles(p)
        
        evt = events[obs][module]['src']['total']
        frequency, power = z_n_search(evt.time, frequencies, nharm=nharm)
        Z2[obs][module] = (frequency, power)
        
        p.line(frequency, power, line_width=1.5)
        
        peak_freq = frequencies[np.argmax(power)]
        peak_power = power[np.argmax(power)]
        
        print(f"Observation: {obs}, module: {module}")
        freq_min, freq_max, power_min, _ = get_frequency_uncertainty(
            frequencies, power, peak_freq, peak_power, nharm, 1
        )
        
        p.vspan(x=[freq_min, freq_max], alpha=0.8, line_color='darkviolet')
        p.hspan(power_min, alpha=0.5, line_color='darkviolet')
        
        plots.append(p)

layout = gridplot([[plots[0], plots[1], plots[2]], [plots[3], plots[4], plots[5]]])
show(layout)

Observation: 0842570101, module: pn
FREQUENCY RESULTS:
  Peak:        3.376764346764347e-02 Hz
  Uncertainty: +1.416361e-05 / -1.416361e-05 Hz

PERIOD RESULTS:
  Peak:        2.961414825876763e+01 s
  Uncertainty: +1.242667e-02 / -1.241625e-02 s
Observation: 0842570101, module: mos1
FREQUENCY RESULTS:
  Peak:        3.377155067155067e-02 Hz
  Uncertainty: +1.465201e-05 / -1.599512e-05 Hz

PERIOD RESULTS:
  Peak:        2.961072204606835e+01 s
  Uncertainty: +1.403108e-02 / -1.284124e-02 s
Observation: 0842570101, module: mos2
FREQUENCY RESULTS:
  Peak:        3.377814407814408e-02 Hz
  Uncertainty: +1.404151e-05 / -1.575092e-05 Hz

PERIOD RESULTS:
  Peak:        2.960494210950575e+01 s
  Uncertainty: +1.381137e-02 / -1.230161e-02 s
Observation: 0902500101, module: pn
FREQUENCY RESULTS:
  Peak:        3.377301587301588e-02 Hz
  Uncertainty: +3.174603e-06 / -2.930403e-06 Hz

PERIOD RESULTS:
  Peak:        2.960943742068900e+01 s
  Uncertainty: +2.569362e-03 / -2.782973e-03 s
Observation:

## Folded Profiles

Phase-folded lightcurves with background subtracted using the area scaling factors. The PN detector provides the best statistics for pulse profile analysis.

In [8]:
ephemeris = 58780
P_orb = 29.60968584
n_bins = 12

# Background scaling factors
bkg_factors = {
    '0842570101': (27**2)/(357**2),
    '0902500101': (20**2)/(113**2)
}

# Fold lightcurves
folded_lightcurves = {}
for obs in lightcurves:
    folded_lightcurves[obs] = {}
    for module in lightcurves[obs]:
        folded_lightcurves[obs][module] = {}
        for region in lightcurves[obs][module]:
            folded_lightcurves[obs][module][region] = {}
            for energy in events[obs][module]['src']:
                folded_lightcurves[obs][module][region][energy] = fold(
                    events[obs][module][region][energy], P_orb, ephemeris, n_bins
                )

# Background subtraction
for obs in folded_lightcurves:
    factor = bkg_factors[obs]
    for module in folded_lightcurves[obs]:
        for energy in folded_lightcurves[obs][module]['src']:
            _, _, bkg_cr, bkg_err, _ = folded_lightcurves[obs][module]['bkg'][energy]
            phase, counts, cr, err, other = folded_lightcurves[obs][module]['src'][energy]
            new_cr = cr - bkg_cr * factor
            new_err = np.sqrt(err**2 + (bkg_err * factor)**2)
            folded_lightcurves[obs][module]['src'][energy] = (phase, counts, new_cr, new_err, other)

In [9]:
# Plot folded lightcurves with hardness ratios
HR = {}
HR_error = {}
plots = []

for obs in folded_lightcurves:
    HR[obs] = {}
    HR_error[obs] = {}
    
    p = figure(width=500, height=360,
               x_axis_label='Phase',
               y_axis_label='Count Rate (cnts/s)',
               min_border=60)
    set_axis_styles(p)
    
    phase, _, cr, err, _ = folded_lightcurves[obs]['pn']['src']['total']
    _, _, soft_cr, soft_err, _ = folded_lightcurves[obs]['pn']['src']['soft']
    _, _, hard_cr, hard_err, _ = folded_lightcurves[obs]['pn']['src']['hard']
    
    plot_folded_lightcurve(p, phase, cr, err, n_bins, 
                           color='firebrick', match_axis_color=True, scale_factor=2)
    
    HR[obs], HR_error[obs] = plot_hardness_ratio(
        p, phase, soft_cr, soft_err, hard_cr, hard_err, n_bins, scale_factor=2
    )
    
    plots.append(p)

plots[0].title = Title(text="2019 XMM", text_font_size="13pt", align="center")
plots[1].title = Title(text="2022 XMM", text_font_size="13pt", align="center")

show(row(plots))

## Statistical Tests

### F-test for Hardness Ratio Variability

Testing whether the hardness ratio varies sinusoidally with pulse phase. The null hypothesis assumes constant HR; the alternative fits a sinusoid. A significant F-statistic indicates genuine spectral variability with phase.

### Phase Correlation Test

Determines whether HR is correlated or anticorrelated with flux (i.e., does the source harden when brighter or fainter?).

In [10]:
observations = ['0842570101', '0902500101']
obs_names = ['2019', '2022']

print("F-TEST FOR HARDNESS RATIO VARIABILITY\n")

for obs, obs_name in zip(observations, obs_names):
    print(f"OBSERVATION: {obs_name}")
    result = test_hardness_ratio_variability(HR[obs], HR_error[obs])
    print()

F-TEST FOR HARDNESS RATIO VARIABILITY

OBSERVATION: 2019

Model 1 (Constant): chi2 = 9.64, dof = 11, chi2_red = 0.88
Model 2 (Sinusoid): chi2 = 4.52, dof = 9, chi2_red = 0.50
  Offset: -0.5346 +/- 0.0122
  Amplitude: 0.0387 +/- 0.0171
  Phase shift: 0.047 cycles

F-test: F = 5.10, p = 3.31e-02, 2.1 sigma
Result: Sinusoid better (p < 0.05)

OBSERVATION: 2022

Model 1 (Constant): chi2 = 5.54, dof = 11, chi2_red = 0.50
Model 2 (Sinusoid): chi2 = 1.65, dof = 9, chi2_red = 0.18
  Offset: -0.6652 +/- 0.0062
  Amplitude: 0.0172 +/- 0.0087
  Phase shift: -0.070 cycles

F-test: F = 10.62, p = 4.28e-03, 2.9 sigma
Result: Sinusoid better (p < 0.05)



In [11]:
print("PHASE CORRELATION TEST: HARDNESS RATIO vs FLUX\n")

for obs, obs_name in zip(observations, obs_names):
    print(f"OBSERVATION: {obs_name}")
    _, _, flux_data, flux_errors, _ = folded_lightcurves[obs]['pn']['src']['total']
    result = test_phase_correlation(flux_data, flux_errors, HR[obs], HR_error[obs])
    print()

PHASE CORRELATION TEST: HARDNESS RATIO vs FLUX

OBSERVATION: 2019

HR amplitude: 0.0387 +/- 0.0171 (2.3 sigma)
Flux amplitude: 0.0829 +/- 0.0065 (12.8 sigma)

Flux max at phase: 0.972 +/- 0.0124
HR max at phase: 0.203 +/- 0.0711
Phase difference: 0.231 +/- 0.0721 cycles

Deviation from in-phase (delta_phi=0): 3.2 sigma
Deviation from anticorr (|delta_phi|=0.5): 3.7 sigma
Result: Intermediate phase shift

OBSERVATION: 2022

HR amplitude: 0.0172 +/- 0.0087 (2.0 sigma)
Flux amplitude: 0.0779 +/- 0.0034 (23.0 sigma)

Flux max at phase: 0.000 +/- 0.0070
HR max at phase: 0.320 +/- 0.0802
Phase difference: 0.319 +/- 0.0805 cycles

Deviation from in-phase (delta_phi=0): 4.0 sigma
Deviation from anticorr (|delta_phi|=0.5): 2.2 sigma
Result: ANTICORRELATED - HR max aligns with flux min

